In [1]:
import pandas as pd

df = pd.read_csv('sevennet_slopes.csv')
df.loc[df['v1_Li_slope'] <= 0, 'v1_Li_slope'] = 1e-15


from dataset import build_dataset

dataset = build_dataset(df, temp = 1000)

[MemorizedFunc(func=<function query_mpid_structure at 0x7d0e0d882520>, location=./cache/joblib)]: Exception while loading results for query_mpid_structure(mpid=[ 'mp-736',
  'mp-1211296',
  'mp-6799',
  'mp-655591',
  'mp-694935',
  'mp-28593',
  'mp-1184020',
  'mp-1110899',
  'mp-1111845',
  'mp-1001069',
  'mp-760046',
  'mp-775430',
  'mp-1222404',
  'mp-14591',
  'mp-554782',
  'mp-30301',
  'mp-1211324',
  'mp-676109',
  'mp-677490',
  'mp-1095543',
  'mp-1110713',
  'mp-1114027',
  'mp-768150',
  'mp-1211446',
  'mp-6604',
  'mp-22905',
  'mp-5614',
  'mp-696128',
  'mp-674531',
  'mp-10620',
  'mp-33526',
  'mp-7117',
  'mp-676650',
  'mp-1222669',
  'mp-1040451',
  'mp-1110709',
  'mp-1110708',
  'mp-567967',
  'mp-1040450',
  'mp-1210789',
  'mp-775806',
  'mp-1222392',
  'mp-641703',
  'mp-950995',
  'mp-1103940',
  'mp-1222589',
  'mp-1...)
 Traceback (most recent call last):
  File "/home/container-user/.local/lib/python3.11/site-packages/joblib/memory.py", line 514, in _c

PES fingerprint cache path set to "./cache". To change, set PFP_CACHE environment variable.
________________________________________________________________________________
[Memory] Calling utils.query_mpid_structure...
query_mpid_structure(mpid=[ 'mp-736',
  'mp-1211296',
  'mp-6799',
  'mp-655591',
  'mp-694935',
  'mp-28593',
  'mp-1184020',
  'mp-1110899',
  'mp-1111845',
  'mp-1001069',
  'mp-760046',
  'mp-775430',
  'mp-1222404',
  'mp-14591',
  'mp-554782',
  'mp-30301',
  'mp-1211324',
  'mp-676109',
  'mp-677490',
  'mp-1095543',
  'mp-1110713',
  'mp-1114027',
  'mp-768150',
  'mp-1211446',
  'mp-6604',
  'mp-22905',
  'mp-5614',
  'mp-696128',
  'mp-674531',
  'mp-10620',
  'mp-33526',
  'mp-7117',
  'mp-676650',
  'mp-1222669',
  'mp-1040451',
  'mp-1110709',
  'mp-1110708',
  'mp-567967',
  'mp-1040450',
  'mp-1210789',
  'mp-775806',
  'mp-1222392',
  'mp-641703',
  'mp-950995',
  'mp-1103940',
  'mp-1222589',
  'mp-1...)


Retrieving SummaryDoc documents:   0%|          | 0/179 [00:00<?, ?it/s]

/home/container-user/.local/lib/python3.11/site-packages/joblib/_store_backends.py:206: FutureWarning: Unable to cache to disk: failed to pickle output. In version 1.5 this will raise an exception. Exception: Can't pickle <class 'mp_api.client.core.client.MPDataDoc'>: it's not found as mp_api.client.core.client.MPDataDoc.
  warnings.warn(


_____________________________________________query_mpid_structure - 5.5s, 0.1min


100%|██████████| 179/179 [00:00<00:00, 933.99it/s]


In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

train_indices, val_indices = train_test_split(np.arange(len(dataset)), test_size=0.2, random_state=42)

train_dataset = [dataset[i] for i in train_indices]
val_dataset = [dataset[i] for i in val_indices]

batch_size = 4

In [3]:
from torch_geometric.loader import DataLoader
from dataset import AtomsToGraphCollater

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
train_dataloader.collate_fn = AtomsToGraphCollater(cutoff = 5, noise_std=0.05, dataset='train')

val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
val_dataloader.collate_fn = AtomsToGraphCollater(cutoff = 5, noise_std=0, dataset='validation')

In [4]:
from train import train

In [ ]:
import torch.optim as optim
import torch.nn as nn

from nn import SimplePeriodicNetwork

radial_cutoff = 5

net = SimplePeriodicNetwork(
    irreps_in="1x1o",  
    irreps_out="1x0e",  # Single scalar (L=0 and even parity) to output (for example) energy
    max_radius=radial_cutoff, # Cutoff radius for convolution
    num_neighbors=10.0,  # scaling factor based on the typical number of neighbors
    pool_nodes=True,  # We pool nodes to predict total energy
)

criterion = nn.MSELoss()  # Example: Mean Squared Error
optimizer = optim.Adam(net.parameters(), lr=0.001)

num_epochs = 50

train(net, train_dataloader, val_dataloader, optimizer, criterion, num_epochs)

100%|██████████| 36/36 [04:22<00:00,  7.29s/it]
